In [ ]:
from ALLCools.mcds import MCDS
from ALLCools.plot import *
from ALLCools.integration import calculate_overlap_score
from wmb import *

import pathlib
import pandas as pd
import numpy as np
import anndata
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [ ]:
dataset = 'AIBS_SMART'
categorical_key = ['L1_annot', 'L2', 'DissectionRegion']

## Load

In [ ]:
mc_annot = cemba.get_mc_annot()

In [ ]:
atac_annot = cemba_atac.get_atac_annot()

In [ ]:
adata_merge = anndata.read_h5ad('final_with_coords.h5ad')

In [ ]:
adata_merge

In [ ]:
mc_adata = adata_merge[adata_merge.obs['Modality'] == 'mC'].copy()
atac_adata = adata_merge[adata_merge.obs['Modality'] == 'ATAC'].copy()

In [ ]:
atac_meta = adata_merge.obs[adata_merge.obs['Modality'] == 'ATAC'].copy()
mc_meta = adata_merge.obs[adata_merge.obs['Modality'] == 'mC'].copy()

## Scatter

In [ ]:
fig, axes = plt.subplots(figsize=(8, 4 * len(categorical_key)),
                         dpi=300,
                         ncols=2,
                         nrows=len(categorical_key))

for ax_row, cat_key in zip(axes, categorical_key):
    ax = ax_row[0]
    if atac_meta[cat_key].unique(
    ).size > 30 or atac_meta[f'{cat_key}_transfer'].unique().size > 30:
        text_anno = False
    else:
        text_anno = True

    categorical_scatter(ax=ax,
                        data=atac_adata,
                        coord_base='tsne',
                        palette='tab20',
                        text_anno=atac_meta[cat_key] if text_anno else None,
                        hue=atac_meta[cat_key],
                        max_points=None)
    ax.set(title=cat_key)

    ax = ax_row[1]
    categorical_scatter(
        ax=ax,
        data=atac_adata,
        coord_base='tsne',
        palette='tab20',
        text_anno=atac_meta[f'{cat_key}_transfer'] if text_anno else None,
        hue=atac_meta[f'{cat_key}_transfer'],
        max_points=None)
    ax.set(title=f'{cat_key} Transfer')

## Confusion Matrix Based On Label Transfer

In [ ]:
for cat_key in categorical_key:
    label_transfer = pd.read_hdf(f'{cat_key}_transfer.hdf')
    label_confusion = label_transfer.groupby(
        atac_annot[cat_key].to_pandas().astype(str)).mean()
    label_confusion.to_hdf(f'{cat_key}.label_transfer.confusion.hdf', key='data')
    if label_confusion.shape[1] == 1 or label_confusion.shape[0] == 1:pass
    else:g = sns.clustermap(data=label_confusion, vmax=0.5)

## Confusion Matrix Based On Co-clustering Overlap Score

In [ ]:
atac_meta['cocluster'] = adata_merge.obs['leiden']
mc_meta['cocluster'] = adata_merge.obs['leiden']

In [ ]:
for cat_key in categorical_key:
    os_mat = calculate_overlap_score(atac_meta[[cat_key, 'leiden']],
                                     mc_meta[[cat_key, 'leiden']])
    os_mat.to_hdf(f'{cat_key}.overlap_score.hdf', key='data')
    if os_mat.shape[1] == 1 or os_mat.shape[0] == 1:pass
    else:g = sns.clustermap(data=os_mat, vmax=0.5)

In [ ]:
!touch finish